In [2]:
import os, sys
import glob, json
import pandas as pd
from tqdm import tqdm

# Add repo
git_dir = os.path.abspath('../')
sys.path.append(os.path.join(git_dir, 'lib', 'utils'))

# Define data path
data_path = os.path.join(git_dir, 'data', 'hansard-parsed')

# Get all json files
data_path_senate = os.path.join(data_path, 'senate')
data_path_hofreps = os.path.join(data_path, 'hofreps')

json_files_senate = [y for x in os.walk(data_path_senate) for y in glob.glob(os.path.join(x[0], '*.json'))]
json_files_hofreps = [y for x in os.walk(data_path_hofreps) for y in glob.glob(os.path.join(x[0], '*.json'))]

def load_jsons(json_list):
    data = []
    for js_path in tqdm(json_list):
        with open(js_path) as f:
            data += json.load(f)
            
    return data

In [3]:
# Create single dataframe
df_senate = pd.DataFrame(load_jsons(json_files_senate))
df_senate = df_senate.dropna(subset=['sentences'])
df_senate['chamber'] = 'Senate'


df_hofreps = pd.DataFrame(load_jsons(json_files_hofreps))
df_hofreps = df_hofreps.dropna(subset=['sentences'])
df_hofreps['chamber'] = 'HouseOfReps'

df = df_senate.append(df_hofreps).reset_index(drop=True).copy()

100%|██████████| 562/562 [00:02<00:00, 205.44it/s]


In [3]:
# Drop entries without sentences
print(len(df))
df = df.dropna(subset=['sentences'])
print(len(df))

163428
161961


In [4]:
def batch_gen(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [5]:
# Add repo
git_dir = os.path.abspath('../')
sys.path.append(os.path.join(git_dir, 'lib', 'GoEmotions-pytorch') )
sys.path.append(os.path.join(git_dir, 'lib', 'utils') )
from transformers import BertTokenizer
from model import BertForMultiLabelClassification
from multilabel_pipeline import MultiLabelPipeline
import re
from tqdm import tqdm
import numpy as np 
import pandas as pd
import pickle

tokenizer = BertTokenizer.from_pretrained("monologg/bert-base-cased-goemotions-ekman")
model = BertForMultiLabelClassification.from_pretrained("monologg/bert-base-cased-goemotions-ekman")
model = model.to('cuda:0')

goemotions = MultiLabelPipeline(
    model=model,
    tokenizer=tokenizer,
    threshold=0.3,
    device=0
)


In [6]:
processed_sentence_lists = []
for ind, sentence_list_raw in enumerate(tqdm(df_senate['sentences'])):
    processed_sentences = []
    sentence_list = []
    for sentence_ind, sentence in enumerate(sentence_list_raw):
        tokens = tokenizer(sentence)
        if len(tokens['input_ids'])>500:
            print('Sentence %d in document %d is too long'%(sentence_ind, ind ))
        else:
            sentence_list.append(sentence)
    
    for batch in batch_gen(sentence_list, n=32):
        processed_sentences.extend(goemotions(batch))
        #processed_sentences.append(goemotions(sentence)[0])
            #ts['emotions'].append(goemotions(sentence)[0])
    processed_sentence_lists.append(processed_sentences)
        

  0%|          | 268/75610 [00:09<27:40, 45.38it/s]

Sentence 0 in document 261 is too long


  0%|          | 324/75610 [00:14<2:25:23,  8.63it/s]

Sentence 14 in document 323 is too long


  2%|▏         | 1374/75610 [00:57<38:34, 32.07it/s]  

...

 99%|█████████▉| 74858/75610 [52:01<01:29,  8.36it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (555 > 512). Running this sequence through the model will result in indexing errors


Sentence 3 in document 74860 is too long


 99%|█████████▉| 74972/75610 [52:05<00:54, 11.78it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (672 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (669 > 512). Running this sequence through the model will result in indexing errors


Sentence 185 in document 74975 is too long
Sentence 187 in document 74975 is too long


100%|██████████| 75610/75610 [52:34<00:00, 23.97it/s]


In [7]:
df_senate_processed = df_senate.copy()
df_senate_processed['sentences_emotions'] = processed_sentence_lists


# Save
df_senate_processed.to_pickle( os.path.join(data_path, 'hansard-senate-emotions-ekman.pkl') )